In [1]:
#!pip install tqdm

In [1]:
import torch
from torch.utils.data import Dataset
import h5py
import numpy as np
from PIL import Image
from utils import read_mat_file, visualize_sample_dataset
from dataloader import SVHDDataset
import scipy.io

from torch.utils.data import Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from model import VGG
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from tqdm import tqdm
from torch.utils.data import DataLoader, random_split
import os
from torchvision import models, transforms
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms
# Train the model
from model import VGG16
#import cv2 

os.environ['TORCH_NNPACK'] = '0'
#from sklearn.model_selection import train_test_split

In [2]:
train_path = '/data/omscs_datasets/train/'
train_mat_file_path = train_path + 'digitStruct.mat'
test_path = '/data/omscs_datasets/train/'
test_mat_file_path = test_path + 'digitStruct.mat'
device = torch.device("cuda")

In [3]:
train_dataset = SVHDDataset(train_mat_file_path, train_path,mode='train')
#test_dataset = SVHDDataset(test_mat_file_path, test_path,mode='test')
#print(train_dataset[0])
#print(test_dataset[0])

## train ##


In [5]:
train_images = []
train_labels = []
for t in tqdm(train_dataset):
    for tt in t:
        train_images.append(tt['image'])
        train_labels.append(tt['label'])

33402it [03:03, 181.84it/s]          


In [6]:
train_labels[3]

3.0

In [7]:
#visualize_sample_dataset(train_dataset, 30)

In [8]:
train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        #if self.transform:
        #    print('image:', image.shape)
        #    image = self.transform(image)

        # Return the image and label as a dictionary or tuple
        sample = {'image': image, 'label': label}

        return sample

In [9]:
print(len(train_images), len(train_labels))
train_dataset = CustomDataset(train_images, train_labels, transform=train_transform)

73257 73257


In [10]:
# Specify the lengths of the training and testing sets
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size

# Use random_split to split the dataset
train_data, test_data = random_split(train_dataset, [train_size, test_size])
print(f'train_data:{len(train_data)}, test_data:{len(test_data)}')

train_data:58605, test_data:14652


In [11]:
# Step 4: Create a DataLoader for your dataset
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
print(len(train_loader), len(test_loader))

1832 458


In [12]:
def test_accuracy(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(test_loader):
            inputs, labels = batch['image'], batch['label']
            inputs = inputs.to(torch.float32).to(device)
            labels = labels.to(torch.float32).to(device)
            labels = labels.long()
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy

In [13]:
def train(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=5):
    
    best_accuracy = 0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        print('## epoch ##', epoch)
        
        for batch in tqdm(train_loader):
            inputs, labels = batch['image'], batch['label']
            inputs = inputs.to(torch.float32).to(device)
            labels = labels.to(torch.float32).to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            labels = labels.long()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = test_accuracy(model, test_loader, device)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss}, Test Accuracy: {epoch_accuracy}%")
        
        if epoch_accuracy > best_accuracy:
            best_accuracy = epoch_accuracy
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"New best model saved with accuracy: {epoch_accuracy}%")

In [14]:
#model = VGG(num_classes=11).to(device)
model =  VGG16(num_classes=11).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# Train the model
model.to(device)
# train(model, train_loader, criterion, optimizer, num_epochs=5)

VGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 2

In [15]:
train(model, train_loader, test_loader, criterion, optimizer, device, num_epochs=5)

## epoch ## 0


  0%|          | 0/1832 [00:00<?, ?it/s]/root/miniconda3/envs/cv_proj/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 458/458 [00:01<00:00, 449.27it/s]


Epoch 1/5, Loss: 0.7917668134966401, Test Accuracy: 89.31203931203932%
New best model saved with accuracy: 89.31203931203932%
## epoch ## 1


100%|██████████| 458/458 [00:00<00:00, 484.16it/s]


Epoch 2/5, Loss: 0.2982090743152856, Test Accuracy: 91.68031668031668%
New best model saved with accuracy: 91.68031668031668%
## epoch ## 2


...

Epoch 3/5, Loss: 0.22126925977833536, Test Accuracy: 92.46519246519246%
New best model saved with accuracy: 92.46519246519246%
## epoch ## 3


100%|██████████| 458/458 [00:00<00:00, 472.67it/s]


Epoch 4/5, Loss: 0.17374393855797038, Test Accuracy: 92.54026754026754%
New best model saved with accuracy: 92.54026754026754%
## epoch ## 4


100%|██████████| 458/458 [00:00<00:00, 480.49it/s]

Epoch 5/5, Loss: 0.13752820527772766, Test Accuracy: 91.38001638001639%


In [16]:
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>